In [97]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

### **Pre-processing data**

In [98]:
df = pd.read_csv('temp_data.csv')

In [99]:
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


In [100]:
y = df['charges']
X = df.drop('charges', axis = 1)

In [101]:
cols = X.select_dtypes('object').columns

In [102]:
X = pd.get_dummies(X, columns = cols)

In [103]:
X

,age,bmi,children,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,1,0,0,1,0,0,0,1
1,18,33.770,1,0,1,1,0,0,0,1,0
2,28,33.000,3,0,1,1,0,0,0,1,0
3,33,22.705,0,0,1,1,0,0,1,0,0
4,32,28.880,0,0,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1333,50,30.970,3,0,1,1,0,0,1,0,0
1334,18,31.920,0,1,0,1,0,1,0,0,0
1335,18,36.850,0,1,0,1,0,0,0,1,0
1336,21,25.800,0,1,0,1,0,0,0,0,1


In [104]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 20)

In [ ]:
y_train['charges'] = np.log1p(y_train['charges'])

In [105]:
scaler = StandardScaler()
X_train['bmi'] = scaler.fit_transform(X_train[['bmi']])
X_test['bmi'] = scaler.transform(X_test[['bmi']])

In [106]:
X_train['age'] = X_train['age'].apply(lambda x: x // 10)
X_test['age'] = X_train['age'].apply(lambda x: x // 10)

In [107]:
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.15, random_state = 20)

In [108]:
random_forest = RandomForestRegressor()
random_forest.fit(X_tr, y_tr)
pred_val = random_forest.predict(X_val)

In [109]:
mse = mean_squared_error(y_val, pred_val)
rmse2 = np.sqrt(mse)
print(rmse2)

5037.902993332766


In [110]:
xgb = XGBRegressor()
xgb.fit(X_tr, y_tr)
pred_val = xgb.predict(X_val)

In [111]:
mse = mean_squared_error(y_val, pred_val)
rmse2 = np.sqrt(mse)
print(rmse2)

5589.176898329372
